# ================================
# SVM with Python Tutorial and Examples
# ================================

# Preliminaries

This notebook command means that plots will be displayed in this notebook.

In [1]:
%matplotlib inline

This gives us Python 3 style printing if we are using Python 2.

In [2]:
from __future__ import print_function

## Modules to Import

"sklean" is scikit-learn, a popular Python module that implements many machine learning technqiues.  We import the part that implements support vector machines

In [3]:
from sklearn import svm

"numpy" is a module that helps do numerical calculations involving vectors and arrays very quickly

In [4]:
import numpy as np

"matplotlib" is a module for making plots.  The plotting part is in matplotlib.pyplot

In [5]:
import matplotlib.pyplot as plt

## Parameters for Making Plots

Change the string on the right hand side of the instruction below to the directory where you want to put the plot you make.  If you leave it as "", the plots will go in the current working directory.

In [6]:
plots_directory = ""

Change the size of plots to be twice as big as default values.  This helps make sure that the plots have high resolution and are large when displayed in this notebook.  Feel free to play with these values.

In [7]:
plt.rcParams['figure.figsize'] = [12., 8.]

Change the fontsize to be much larger (to go along with larger figure size.)  Feel free to play with this value.

In [8]:
plt.rcParams['font.size'] = 24.

# Exploring Kernels in One Dimensions

## Linearly Seperable Case

We start by considering a collection of 3 points in 1 dimension (1-dimensional feature space) that are linearly separable.

In [9]:
points = np.array([[-1.],[0.],[1.]])
labels = [-1,-1,1]

In [10]:
our_linear_svm = svm.SVC(kernel = 'linear', C = 1.)
our_linear_svm.fit(points, labels)
our_linear_svm.predict(points)

array([-1, -1,  1])

We see that everything works.

## Quadratically Separable Case: Linear and Polynomial Kernels

Now we try **non-linearly** separable points

In [11]:
points = np.array([[-1.],[0.],[1.]])
labels = [1,-1,1]

We start by trying the **linear** kernel (for several choices of C)

In [12]:
our_linear_svm_list = [svm.SVC(kernel = 'linear', C = C) for C in [0.01, 1., 1e2, 1e6]]
[o.fit(points, labels) for o in our_linear_svm_list]
[o.predict(points) for o in our_linear_svm_list]

[array([1, 1, 1]), array([1, 1, 1]), array([1, 1, 1]), array([1, 1, 1])]

We see that we do not reproduce "labels" for any value of C which we try.  (We will discuss the meaning and importance of $C$ more later.)

### **Introduction to the Polynomial Kernel**

Now we try the **polynomial** kernel.  The idea of this kernel is that, e.g.,  
  
$k(x,y) = (x y)^d$  
  
is the (dot) product if our values were $x^d$ and $y^d$ instead of $x$ and $y$

If we are dealing with vectors (in a feature space with dimension, $N$ > 1), then, taking $N = 2$ and $d = 2$ as an example  
  
$({\bf{x}} \cdot {\bf{y}})^2 = (x_1 y_1 + x_2 y_2)^2 = x_1^2 y_1^2 + 2 x_1 x_2 y_1 y_2 + x_2^2 y_2^2$.

This is a dot product in a 3 dimensional space where  
  
${\bf{x}} \to (x_1^2, \sqrt{2} x_1 x_2, x_2^2)$
  
So if our problem is linearly separable in this new space, the SVM will work with that polynomial kernel.  

Note that if our form of the polynomial kernel is $({\bf{x}} \cdot {\bf{y}})^d$, then our new space is all monomials (single term polynomials) of degree exactly $d$.  This is the form used in scikit-learn.  However, if instead we defined our polynomial kernel by $({\bf{x}} \cdot {\bf{y}} + c)^d$, then our "new space" contains all monomials of degree $\le d$.  We can implement this kernel by one of three methods
  
1.  Set the coef0 variable (preferred).  
2.  Pass either a kernel function to the svm (class instance).
3.  Pass a (Gram) matrix with the function evaluating for all pairs of points in the training set.
  
We will implement all three choices in this notebook.  Our implementations of 2. and 3. are to show how to use precomputed and callable kernels rather than as a strategy for including a constant in the polynomial method.

### Polynomial Kernel of Degree 1

The polynomial kernel of degree 1 is the same as the linear kernel, so it will not work.

In [13]:
our_poly_svm_list = [svm.SVC(kernel = 'poly', degree = 1, C = C) for C in [0.01, 0.99, 1., 1.01, 10, 1e2, 1e6]]
[o.fit(points, labels) for o in our_poly_svm_list]
[o.predict(points) for o in our_poly_svm_list]

[array([1, 1, 1]),
 array([1, 1, 1]),
 array([1, 1, 1]),
 array([1, 1, 1]),
 array([1, 1, 1]),
 array([1, 1, 1]),
 array([1, 1, 1])]

We do reproduce the labels ([1,-1,1]) for sufficient values of $C$ with the quadratic kernel.  This is as we expect, since if we square the points values, the point with label $-1$ has value $0$, while the points with value $1$ have value $1$.

### Polynomial Kernel of Degree 2

In [14]:
our_poly_svm_list = [svm.SVC(kernel = 'poly', degree = 2, C = C) for C in [0.01, 0.99, 1., 1.01, 10, 1e2, 1e6]]
[o.fit(points, labels) for o in our_poly_svm_list]
[o.predict(points) for o in our_poly_svm_list]

[array([1, 1, 1]),
 array([1, 1, 1]),
 array([1, 1, 1]),
 array([ 1, -1,  1]),
 array([ 1, -1,  1]),
 array([ 1, -1,  1]),
 array([ 1, -1,  1])]

We now spend some time understanding the role of $C$.

In [15]:
C_list = [0.01, 0.99, 1., 1.01, 2, 10, 1e6]

extended_points = np.array([[-2], [-1], [0], [1], [2]])

our_poly_svm_list = [svm.SVC(kernel = 'poly', degree = 2, C = C) for C in C_list]
[o.fit(points, labels) for o in our_poly_svm_list]

for o, C in zip(our_poly_svm_list, C_list):
    print('C = ', C)
    print(o.decision_function(extended_points))
    print(o.dual_coef_)
    print(o.support_vectors_)
    print(o.intercept_)
    print(o.predict(points), '\n')



C =  0.01
[ 1.03  1.    0.99  1.    1.03]
[[-0.01  0.01]]
[[ 0.]
 [ 1.]]
[ 0.99]
[1 1 1] 

C =  0.99
[ 3.97  1.    0.01  1.    3.97]
[[-0.99  0.99]]
[[ 0.]
 [ 1.]]
[ 0.01]
[1 1 1] 

C =  1.0
[ 4.  1. -0.  1.  4.]
[[-1.  1.]]
[[ 0.]
 [ 1.]]
[-0.]
[1 1 1] 

C =  1.01
[ 4.03  1.   -0.01  1.    4.03]
[[-1.01  1.01]]
[[ 0.]
 [ 1.]]
[-0.01]
[ 1 -1  1] 

C =  2
[ 7.  1. -1.  1.  7.]
[[-2.  2.]]
[[ 0.]
 [ 1.]]
[-1.]
[ 1 -1  1] 

C =  10
[ 7.  1. -1.  1.  7.]
[[-2.  2.]]
[[ 0.]
 [ 1.]]
[-1.]
[ 1 -1  1] 

C =  1000000.0
[ 7.  1. -1.  1.  7.]
[[-2.  2.]]
[[ 0.]
 [ 1.]]
[-1.]
[ 1 -1  1] 



Remember (see the SVM notes) that in SVMs we are solving for values of $\alpha_i$ in the "dual problem".  $C$ gives a maximum absolute value for these $\alpha_i$.   

We see that our two "support vectors are" [0.] and [1.].  
  
  Clearly ${\bf{x}} \cdot $(0) = 0, and  
  
  ${\bf{x}} \cdot $(1) = x.
  
So $\sum_i \alpha_i \Phi({\bf{x}}, {\bf{x}}_i) = \alpha_2 x^2.$

The (KKT) condition our support vectors must satisfy is

$y_i \sum_j \alpha_j \Phi({\bf{x}}_i, {\bf{x}}_j) + b = 1$  
  
  (from generalizing (1.15) in the handout)

For our specific example, we obtain the equations:  
      
$b = -1$  
  
$\alpha_2 (1)^2 + b = 1$

Solving these equations, we obtain 
  
$\alpha_2 = 2$

So we satisfy these criteria only when $C \ge 2$.  However, if we restrict $\alpha_2 \le C$, then we get $\alpha_2 = C$ with $b$ set accordingly.  We can only get a function where $0$ has the correct sign when $b < 0$, i.e., $\alpha_2 (1)^2 > 1$.

### Polynomial Kernel of Degree 3

We see that we also **cannot** reproduce the label list with degree = 3.

In [16]:
our_poly_svm_list = [svm.SVC(kernel = 'poly', degree = 3, C = C) for C in [0.01, 0.99, 1., 1.01, 10, 1e2, 1e6]]
[o.fit(points, labels) for o in our_poly_svm_list]
[o.predict(points) for o in our_poly_svm_list]

[array([1, 1, 1]),
 array([1, 1, 1]),
 array([1, 1, 1]),
 array([1, 1, 1]),
 array([1, 1, 1]),
 array([1, 1, 1]),
 array([1, 1, 1])]

But we can add plug in a coef0 to turn our kernel into  
$K(\mathbf{x}, \mathbf{y}) = (\mathbf{x} \cdot \mathbf{y} + 1)^3$

In [17]:
our_poly_svm_list = [svm.SVC(kernel = 'poly', degree = 3, C = C, coef0=1.) for C in [0.01, 0.99, 1., 1.01, 10, 1e2, 1e6]]
[o.fit(points, labels) for o in our_poly_svm_list]
[o.predict(points) for o in our_poly_svm_list]

[array([1, 1, 1]),
 array([ 1, -1,  1]),
 array([ 1, -1,  1]),
 array([ 1, -1,  1]),
 array([ 1, -1,  1]),
 array([ 1, -1,  1]),
 array([ 1, -1,  1])]

## Quadratically Separable Case: Precomputed and Callable Kernels

### Precomputed

**We start by calculating the degree 2 polynomial kernel explicitly**

In [18]:
gram = np.dot(points, points.T)**2

If we want to consider additional points, we need to calculate the kernel function for these points and our training vevctors.

In [19]:
extended_points

array([[-2],
       [-1],
       [ 0],
       [ 1],
       [ 2]])

In [20]:
more_points_matrix = np.dot(extended_points, points.T)**2

In [21]:
our_pre_svm_list = [svm.SVC(kernel = 'precomputed', C = C) for C in [0.01, 0.99, 1., 1.01, 10, 1e2, 1e6]]
[o.fit(gram, labels) for o in our_pre_svm_list]

for o in our_pre_svm_list:
    print('C = ', o.C)
    print(o.decision_function(more_points_matrix))
    print(o.dual_coef_)
    print(o.intercept_)
    print(o.predict(gram), '\n')

C =  0.01
[ 1.03  1.    0.99  1.    1.03]
[[-0.01  0.01]]
[ 0.99]
[1 1 1] 

C =  0.99
[ 3.97  1.    0.01  1.    3.97]
[[-0.99  0.99]]
[ 0.01]
[1 1 1] 

C =  1.0
[ 4.  1. -0.  1.  4.]
[[-1.  1.]]
[-0.]
[1 1 1] 

C =  1.01
[ 4.03  1.   -0.01  1.    4.03]
[[-1.01  1.01]]
[-0.01]
[ 1 -1  1] 

C =  10
[ 7.  1. -1.  1.  7.]
[[-2.  2.]]
[-1.]
[ 1 -1  1] 

C =  100.0
[ 7.  1. -1.  1.  7.]
[[-2.  2.]]
[-1.]
[ 1 -1  1] 

C =  1000000.0
[ 7.  1. -1.  1.  7.]
[[-2.  2.]]
[-1.]
[ 1 -1  1] 



So we again see that things work for $C > 1$.

**We now obtain success with a modified degree 3 kernel**

In [22]:
gram = (np.dot(points, points.T) + 1)**3

In [23]:
more_points_matrix = np.dot(extended_points, points.T + 1)**3

In [24]:
our_pre_svm_list = [svm.SVC(kernel = 'precomputed', C = C) for C in [0.01, 0.1, 0.5, 1.]]
[o.fit(gram, labels) for o in our_pre_svm_list]

for o in our_pre_svm_list:
    print('C = ', o.C)
    print(o.decision_function(more_points_matrix))
    print(o.dual_coef_)
    print(o.intercept_)
    print(o.predict(gram), '\n')

C =  0.01
[ 0.73  0.94  0.97  1.    1.21]
[[-0.01   0.005  0.005]]
[ 0.97]
[1 1 1] 

C =  0.1
[-1.7  0.4  0.7  1.   3.1]
[[-0.1   0.05  0.05]]
[ 0.7]
[1 1 1] 

C =  0.5
[-12.5  -2.   -0.5   1.   11.5]
[[-0.5   0.25  0.25]]
[-0.5]
[ 1 -1  1] 

C =  1.0
[-16.99296248  -2.99864663  -0.99945865   0.99972933  14.99404518]
[[-0.66639599  0.333198    0.333198  ]]
[-0.99945865]
[ 1 -1  1] 



### Callable

**We pass the SVM a function implementing the degree 2 polynomial kernel.**

In [25]:
def quadratic(X,Y):
    return np.dot(X, Y.T)**2

In [26]:
our_call_svm_list = [svm.SVC(kernel = quadratic, C = C) for C in [0.1, 0.99, 1., 1.01, 2.,10.]]
[o.fit(points, labels) for o in our_call_svm_list]

for o in our_call_svm_list:
    print('C = ', o.C)
    print(o.decision_function(extended_points))
    print(o.dual_coef_)
    print(o.support_vectors_)
    print(o.intercept_)
    print(o.predict(points), '\n')

C =  0.1
[ 1.3  1.   0.9  1.   1.3]
[[-0.1  0.1]]
[]
[ 0.9]
[1 1 1] 

C =  0.99
[ 3.97  1.    0.01  1.    3.97]
[[-0.99  0.99]]
[]
[ 0.01]
[1 1 1] 

C =  1.0
[ 4.  1. -0.  1.  4.]
[[-1.  1.]]
[]
[-0.]
[1 1 1] 

C =  1.01
[ 4.03  1.   -0.01  1.    4.03]
[[-1.01  1.01]]
[]
[-0.01]
[ 1 -1  1] 

C =  2.0
[ 7.  1. -1.  1.  7.]
[[-2.  2.]]
[]
[-1.]
[ 1 -1  1] 

C =  10.0
[ 7.  1. -1.  1.  7.]
[[-2.  2.]]
[]
[-1.]
[ 1 -1  1] 



**We pass the SVM a function implementing a degree 3 polynomial kernel**

In [27]:
def cubic(X,Y):
    return (np.dot(X, Y.T) + 1)**3

In [28]:
our_call_svm_list = [svm.SVC(kernel = cubic, C = C) for C in [0.1, 0.5, 1.,5.]]
[o.fit(points, labels) for o in our_call_svm_list]

for o in our_call_svm_list:
    print('C = ', o.C)
    print(o.decision_function(extended_points))
    print(o.dual_coef_)
    print(o.support_vectors_)
    print(o.intercept_)
    print(o.predict(points), '\n')

C =  0.1
[ 1.9  1.   0.7  1.   1.9]
[[-0.1   0.05  0.05]]
[]
[ 0.7]
[1 1 1] 

C =  0.5
[ 5.5  1.  -0.5  1.   5.5]
[[-0.5   0.25  0.25]]
[]
[-0.5]
[ 1 -1  1] 

C =  1.0
[ 6.99729326  0.99972933 -0.99945865  0.99972933  6.99729326]
[[-0.66639599  0.333198    0.333198  ]]
[]
[-0.99945865]
[ 1 -1  1] 

C =  5.0
[ 6.99729326  0.99972933 -0.99945865  0.99972933  6.99729326]
[[-0.66639599  0.333198    0.333198  ]]
[]
[-0.99945865]
[ 1 -1  1] 



## Quadratically Separable Case: 'rbf' and 'sigmoid' kernels

### rbf

In [29]:
our_rbf_svm_list = [svm.SVC(kernel = 'rbf', C = C) for C in [0.1, 1., 1.25, 1.5, 2., 3., 5., 10.]]
[o.fit(points, labels) for o in our_rbf_svm_list]

for o in our_rbf_svm_list:
    print('C = ', o.C)
    print('gamma = ', o.gamma)
    print(o.decision_function(extended_points))
    print(o.dual_coef_)
    print(o.support_vectors_)
    print(o.intercept_)
    print(o.predict(points), '\n')

C =  0.1
gamma =  auto
[ 1.00244074  1.          0.92266011  1.          1.00244074]
[[-0.1   0.05  0.05]]
[[ 0.]
 [-1.]
 [ 1.]]
[ 0.98587216]
[1 1 1] 

C =  1.0
gamma =  auto
[ 1.02440742  1.00000001  0.22660107  1.00000001  1.02440742]
[[-1.   0.5  0.5]]
[[ 0.]
 [-1.]
 [ 1.]]
[ 0.85872163]
[1 1 1] 

C =  1.25
gamma =  auto
[ 1.03050927  1.00000001  0.03325134  1.00000001  1.03050927]
[[-1.25   0.625  0.625]]
[[ 0.]
 [-1.]
 [ 1.]]
[ 0.82340204]
[1 1 1] 

C =  1.5
gamma =  auto
[ 1.03661113  1.00000001 -0.16009839  1.00000001  1.03661113]
[[-1.5   0.75  0.75]]
[[ 0.]
 [-1.]
 [ 1.]]
[ 0.78808245]
[ 1 -1  1] 

C =  2.0
gamma =  auto
[ 1.04881483  1.00000002 -0.54679786  1.00000002  1.04881483]
[[-2.  1.  1.]]
[[ 0.]
 [-1.]
 [ 1.]]
[ 0.71744326]
[ 1 -1  1] 

C =  3.0
gamma =  auto
[ 1.06311536  0.9998687  -1.00013134  1.00026266  1.06326294]
[[-2.58624227  1.29292048  1.29332179]]
[[ 0.]
 [-1.]
 [ 1.]]
[ 0.63468557]
[ 1 -1  1] 

C =  5.0
gamma =  auto
[ 1.06311536  0.9998687  -1.00013134 

### sigmoid

In [30]:
def sigmoid_kernel(X, Y, gamma, r):
    return np.tanh(gamma * np.dot(X, Y.T) + r)

In [31]:
our_sigmoid_svm_list = [svm.SVC(kernel = 'sigmoid', C = C) for C in [1e-6, 1e-4, 1e-2, 1, 1e2, 1e4, 1e6]]

for o in our_sigmoid_svm_list:
    o.gamma = 1.
    o.coef0 = -1e-2
    o.fit(points, labels)
    print('C = ', o.C)
    print(o.decision_function(extended_points))
    print(o.dual_coef_)
    print(o.support_vectors_)
    print(o.intercept_)
    print(o.predict(points), '\n')

C =  1e-06
[ 0.99999904  0.99999924  0.99999999  1.00000076  1.00000097]
[[ -1.00000000e-06   1.00000000e-06]]
[[ 0.]
 [ 1.]]
[ 0.99999999]
[1 1 1] 

C =  0.0001
[ 0.99990395  0.99992384  0.99999942  1.00007616  1.00009675]
[[-0.0001  0.0001]]
[[ 0.]
 [ 1.]]
[ 0.99999942]
[1 1 1] 

C =  0.01
[ 1.00003493  1.          0.999942    1.          1.00003493]
[[-0.01   0.005  0.005]]
[[ 0.]
 [-1.]
 [ 1.]]
[ 0.999942]
[1 1 1] 

C =  1
[ 1.0034933   1.00000001  0.9942002   1.00000002  1.00349332]
[[-1.          0.49999999  0.50000001]]
[[ 0.]
 [-1.]
 [ 1.]]
[ 0.9942002]
[1 1 1] 

C =  100.0
[ 1.34933122  1.00000156  0.42001961  1.00000159  1.34933126]
[[-100.           49.99999999   50.00000001]]
[[ 0.]
 [-1.]
 [ 1.]]
[ 0.42001961]
[1 1 1] 

C =  10000.0
[ 2.20479391  1.00033389 -0.99967156  0.99934855  2.20354662]
[[-344.75430088  172.3774739   172.37682698]]
[[ 0.]
 [-1.]
 [ 1.]]
[-0.99967156]
[ 1 -1  1] 

C =  1000000.0
[ 2.20479391  1.00033389 -0.99967156  0.99934855  2.20354662]
[[-344.754

In [32]:
our_sigmoid_svm_list = [svm.SVC(kernel = 'precomputed', C = C) for C in [172.4, 172.5, 350, 1e3]]

for o in our_sigmoid_svm_list:
    o.fit(sigmoid_kernel(points, points, 1., -1e-2), labels)
    print('C = ', o.C)
    print(o.decision_function(sigmoid_kernel(extended_points, points,1., -1e-2)))
    print(o.dual_coef_)
    print(o.support_vectors_)
    print(o.intercept_)
    print(o.predict(sigmoid_kernel(points, points, 1., -1e-2)), '\n')

C =  172.4
[  1.60224708e+00   1.00000273e+00   1.13810599e-04   1.00000271e+00
   1.60224705e+00]
[[-172.4          86.20000001   86.19999999]]
[]
[ 0.00011381]
[1 1 1] 

C =  172.5
[  1.60259641e+00   1.00000273e+00  -4.66169789e-04   1.00000271e+00
   1.60259638e+00]
[[-172.5          86.25000001   86.24999999]]
[]
[-0.00046617]
[ 1 -1  1] 

C =  350
[ 2.20479391  1.00033389 -0.99967156  0.99934855  2.20354662]
[[-344.75430088  172.3774739   172.37682698]]
[]
[-0.99967156]
[ 1 -1  1] 

C =  1000.0
[ 2.20479391  1.00033389 -0.99967156  0.99934855  2.20354662]
[[-344.75430088  172.3774739   172.37682698]]
[]
[-0.99967156]
[ 1 -1  1] 



In [33]:
kernel = lambda X, Y: sigmoid_kernel(X, Y, 1., -1e-2)
our_sigmoid_svm_list = [svm.SVC(kernel = kernel, C = C) for C in [172.4, 172.5, 350, 1e3]]

for o in our_sigmoid_svm_list:
    o.fit(points, labels)
    print('C = ', o.C)
    print(o.decision_function(extended_points))
    print(o.dual_coef_)
    print(o.support_vectors_)
    print(o.intercept_)
    print(o.predict(points), '\n')

C =  172.4
[  1.60224708e+00   1.00000273e+00   1.13810599e-04   1.00000271e+00
   1.60224705e+00]
[[-172.4          86.20000001   86.19999999]]
[]
[ 0.00011381]
[1 1 1] 

C =  172.5
[  1.60259641e+00   1.00000273e+00  -4.66169789e-04   1.00000271e+00
   1.60259638e+00]
[[-172.5          86.25000001   86.24999999]]
[]
[-0.00046617]
[ 1 -1  1] 

C =  350
[ 2.20479391  1.00033389 -0.99967156  0.99934855  2.20354662]
[[-344.75430088  172.3774739   172.37682698]]
[]
[-0.99967156]
[ 1 -1  1] 

C =  1000.0
[ 2.20479391  1.00033389 -0.99967156  0.99934855  2.20354662]
[[-344.75430088  172.3774739   172.37682698]]
[]
[-0.99967156]
[ 1 -1  1] 

